<a href="https://colab.research.google.com/github/Solrak97/tf-vgg16-flowerdata/blob/main/VGG_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transferencia de aprendizaje utilizando VGG16

In [1]:
%%capture
! pip install kaggle torchmetrics wandb
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! rm -rf flowers
! kaggle datasets download alxmamaev/flowers-recognition
! unzip flowers-recognition.zip
! rm -r flowers-recognition.zip
! rm -rf flowers/sample_data/

In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch import nn
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
import pickle

from torchmetrics import F1Score, Recall, Precision, Accuracy
import wandb

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Carga del dataset

In [3]:
transform_train = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
                                      transforms.ColorJitter(brightness=1, contrast=1, saturation=1),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])


transform = transforms.Compose([transforms.Resize((224,224)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])

dataset = ImageFolder("flowers")
train, val = torch.utils.data.random_split(dataset, [4317 - 863, 863], generator=torch.Generator().manual_seed(42))

train.dataset.transform = transform_train
val.dataset.transform = transform

training_loader = torch.utils.data.DataLoader(train, batch_size=200, shuffle=True)
validation_loader = torch.utils.data.DataLoader(val, batch_size = 200, shuffle=False)

In [4]:
def im_convert(tensor):
  image = tensor.cpu().clone().detach().numpy()
  image = image.transpose(1, 2, 0)
  image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
  image = image.clip(0, 1)
  return image

## Modificación del modelo base VGG16

In [5]:
classes = ('margaritas', 'rosas', 'dientes de león', 'tulipanes', 'girasoles')

In [6]:
# Creación del modelo VGG16
model = models.vgg16(pretrained=True)

# Congelamiento del modelo base
for param in model.features.parameters():
  param.requires_grad = False

n_inputs = model.classifier[6].in_features
last_layer = nn.Linear(n_inputs, len(classes))

# Reemplzo de la capa de salida
model.classifier[6] = last_layer
model = model.to(device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Entrenamiento del modelo

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)


wandb.init(project="Clasificador de plantas")

wandb: Currently logged in as: solrak (panas). Use `wandb login --relogin` to force relogin


In [8]:
epochs = 10
running_loss_history = []
running_corrects_history = []
val_running_loss_history = []
val_running_corrects_history = []


f1 = F1Score(num_classes=5).to(device)
recall = Recall(average='macro', num_classes=5).to(device)
precision = Precision(average='macro', num_classes=5).to(device)
accuracy = Accuracy().to(device)


for e in range(epochs):
  
  running_loss = 0.0
  running_corrects = 0.0
  val_running_loss = 0.0
  val_running_corrects = 0.0
  
  for inputs, labels in training_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    _, preds = torch.max(outputs, 1)
    running_loss += loss.item()
    running_corrects += torch.sum(preds == labels.data)

  else:
    with torch.no_grad():
      for val_inputs, val_labels in validation_loader:
        val_inputs = val_inputs.to(device)
        val_labels = val_labels.to(device)
        val_outputs = model(val_inputs)
        val_loss = criterion(val_outputs, val_labels)

        _, val_preds = torch.max(val_outputs, 1)
        val_running_loss += val_loss.item()
        val_running_corrects += torch.sum(val_preds == val_labels.data)

        wandb.log({'Validation Accuracy': accuracy(val_outputs, val_labels), 'F1 Score': f1(
          val_outputs, val_labels), 'Recall': recall(val_outputs, val_labels), 'Precision': precision(val_outputs, val_labels)})

        print({'Validation Accuracy': accuracy(val_outputs, val_labels), 'F1 Score': f1(
          val_outputs, val_labels), 'Recall': recall(val_outputs, val_labels), 'Precision': precision(val_outputs, val_labels)})
    
      
    epoch_loss = running_loss/len(training_loader.dataset)
    epoch_acc = running_corrects.float()/ len(training_loader.dataset)
    running_loss_history.append(epoch_loss)
    running_corrects_history.append(epoch_acc)
    
    val_epoch_loss = val_running_loss/len(validation_loader.dataset)
    val_epoch_acc = val_running_corrects.float()/ len(validation_loader.dataset)
    val_running_loss_history.append(val_epoch_loss)
    val_running_corrects_history.append(val_epoch_acc)


{'Validation Accuracy': tensor(0.8500, device='cuda:0'), 'F1 Score': tensor(0.8500, device='cuda:0'), 'Recall': tensor(0.8425, device='cuda:0'), 'Precision': tensor(0.8571, device='cuda:0')}
{'Validation Accuracy': tensor(0.8350, device='cuda:0'), 'F1 Score': tensor(0.8350, device='cuda:0'), 'Recall': tensor(0.8366, device='cuda:0'), 'Precision': tensor(0.8340, device='cuda:0')}
{'Validation Accuracy': tensor(0.8250, device='cuda:0'), 'F1 Score': tensor(0.8250, device='cuda:0'), 'Recall': tensor(0.8166, device='cuda:0'), 'Precision': tensor(0.8167, device='cuda:0')}
{'Validation Accuracy': tensor(0.8500, device='cuda:0'), 'F1 Score': tensor(0.8500, device='cuda:0'), 'Recall': tensor(0.8382, device='cuda:0'), 'Precision': tensor(0.8419, device='cuda:0')}
{'Validation Accuracy': tensor(0.8730, device='cuda:0'), 'F1 Score': tensor(0.8730, device='cuda:0'), 'Recall': tensor(0.8660, device='cuda:0'), 'Precision': tensor(0.8944, device='cuda:0')}
{'Validation Accuracy': tensor(0.8750, device

## Guardar modelo

In [9]:
def save_checkpoint(model, path):
    # Basic details
    checkpoint = {
        'class_to_idx': model.class_to_idx,
        'idx_to_class': model.idx_to_class,
        'epochs': model.epochs,
    }

    # Extract the final classifier and the state dictionary
    
    # Check to see if model was parallelized
    if torch.cuda.is_available():
      if torch.cuda.device_count() > 1:
        checkpoint['classifier'] = model.module.classifier
        checkpoint['state_dict'] = model.module.state_dict()

    else:
       checkpoint['classifier'] = model.classifier
       checkpoint['state_dict'] = model.state_dict()

    # Add the optimizer
    checkpoint['optimizer'] = model.optimizer
    checkpoint['optimizer_state_dict'] = model.optimizer.state_dict()

    # Save the data to the path
    torch.save(checkpoint, path)

## Cargar modelo

In [10]:
def load_checkpoint(path, device):

    # Load in checkpoint
    checkpoint = torch.load(path)

    
    model = models.vgg16(pretrained=True)
    # Make sure to set parameters as not trainable
    for param in model.parameters():
        param.requires_grad = False
    model.classifier = checkpoint['classifier']


    # Load in the state dict
    model.load_state_dict(checkpoint['state_dict'])

    total_params = sum(p.numel() for p in model.parameters())
    print(f'{total_params:,} total parameters.')
    total_trainable_params = sum(
        p.numel() for p in model.parameters() if p.requires_grad)
    print(f'{total_trainable_params:,} total gradient parameters.')

    # Move to device
    model = model.to(device)

    # Model basics
    model.class_to_idx = checkpoint['class_to_idx']
    model.idx_to_class = checkpoint['idx_to_class']
    model.epochs = checkpoint['epochs']

    # Optimizer
    optimizer = checkpoint['optimizer']
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    return model, optimizer

In [11]:
save_checkpoint(model, 'classifier.trch')
load_checkpoint('classifier.trch', device)

AttributeError: ignored